In [3]:
import itertools
from tabulate import tabulate

# ANSI escape codes for bold formatting.
BOLD_START = "\033[1m"
BOLD_END = "\033[0m"

def poly_to_str(poly, var="x"):
    """
    Convert a polynomial (given as a tuple or list of coefficients, lowest degree first)
    into a human-readable string.
    """
    terms = []
    for i, coeff in enumerate(poly):
        if coeff == 0:
            continue
        if i == 0:
            term = f"{coeff}"
        elif i == 1:
            term = f"{var}" if coeff == 1 else f"{coeff}{var}"
        else:
            term = f"{var}^{i}" if coeff == 1 else f"{coeff}{var}^{i}"
        terms.append(term)
    if not terms:
        return "0"
    return "+".join(reversed(terms))

def poly_mul(a, b, p):
    """
    Multiply two polynomials a and b with coefficients in Z_p.
    Both a and b are lists (lowest degree first).
    Returns a list representing the product.
    """
    result = [0] * (len(a) + len(b) - 1)
    for i, coeff_a in enumerate(a):
        for j, coeff_b in enumerate(b):
            result[i+j] = (result[i+j] + coeff_a * coeff_b) % p
    return result

def poly_mod(poly, mod_poly, p):
    """
    Reduce a polynomial modulo mod_poly over Z_p.
    poly and mod_poly are lists of coefficients (lowest degree first).
    Assumes mod_poly is monic.
    """
    poly = poly[:]  # work on a copy
    while len(poly) >= len(mod_poly):
        diff = len(poly) - len(mod_poly)
        lead_coeff = poly[-1]
        if lead_coeff != 0:
            for i in range(len(mod_poly)):
                poly[i + diff] = (poly[i + diff] - lead_coeff * mod_poly[i]) % p
        while poly and poly[-1] == 0:
            poly.pop()
    return poly if poly else [0]

def multiply(a, b, p, mod_poly):
    """
    Multiply two elements (represented as lists of coefficients of length n)
    in Z_p[x]/(mod_poly). Returns the resulting element as a tuple (of length n).
    """
    prod = poly_mul(a, b, p)
    rem = poly_mod(prod, mod_poly, p)
    n = len(mod_poly) - 1
    if len(rem) < n:
        rem = rem + [0] * (n - len(rem))
    return tuple(rem[:n])

def print_multiplication_table_tabulate(p, mod_poly, var="x"):
    """
    Prints the multiplication table for the quotient ring Z_p[x] / (mod_poly)
    in a neat grid format using the tabulate library.
    The header row and the first column are printed in bold.

    Parameters:
      - p: a prime number (the modulus for the coefficients)
      - mod_poly: the modulus polynomial, given as a list of coefficients
                  [constant, coeff_of_x, coeff_of_x^2, ..., coeff_of_x^n]
                  (It is assumed that mod_poly is monic.)
      - var: the variable name to use in printing the polynomial (default "x")
    """
    n = len(mod_poly) - 1  # Degree of the modulus polynomial
    elements = [tuple(poly) for poly in itertools.product(range(p), repeat=n)]
    names = {elem: poly_to_str(elem, var) for elem in elements}

    # Create headers with bold formatting: first cell is blank and others are bold.
    header = [BOLD_START + "" + BOLD_END]
    header.extend([BOLD_START + names[e] + BOLD_END for e in elements])

    table = []
    for e in elements:
        # Bold the first column of each row.
        row = [BOLD_START + names[e] + BOLD_END]
        for f in elements:
            prod = multiply(list(e), list(f), p, mod_poly)
            row.append(names[prod])
        table.append(row)

    mod_poly_str = poly_to_str(mod_poly, var)
    print(f"Multiplication Table for Z_{p}[{var}] / ({mod_poly_str}):")
    print(tabulate(table, headers=header, tablefmt="grid"))

# Example usage:
# For GF(3)[x]/(2+ x + 0*x^2+ 0*x^3+ 0*x^4+ x^5), where mod_poly = [2, 1, 0, 0, 0, 1]
print_multiplication_table_tabulate(2, [ 1, 0, 0, 1], var="x")


Multiplication Table for Z_2[x] / (x^3+1):
+---------+-----+---------+---------+---------+---------+---------+-------+-----------+
|         |   0 | x^2     | x       | x^2+x   | 1       | x^2+1   | x+1   | x^2+x+1   |
+=========+=====+=========+=========+=========+=========+=========+=======+===========+
| 0       |   0 | 0       | 0       | 0       | 0       | 0       | 0     | 0         |
+---------+-----+---------+---------+---------+---------+---------+-------+-----------+
| x^2     |   0 | x       | 1       | x+1     | x^2     | x^2+x   | x^2+1 | x^2+x+1   |
+---------+-----+---------+---------+---------+---------+---------+-------+-----------+
| x       |   0 | 1       | x^2     | x^2+1   | x       | x+1     | x^2+x | x^2+x+1   |
+---------+-----+---------+---------+---------+---------+---------+-------+-----------+
| x^2+x   |   0 | x+1     | x^2+1   | x^2+x   | x^2+x   | x^2+1   | x+1   | 0         |
+---------+-----+---------+---------+---------+---------+---------+-------+--

In [2]:
import itertools
from tabulate import tabulate

# ANSI escape codes for bold formatting.
BOLD_START = "\033[1m"
BOLD_END = "\033[0m"

def poly_to_str(poly, var="x"):
    """
    Convert a polynomial (given as a tuple or list of coefficients, lowest degree first)
    into a human-readable string.
    """
    terms = []
    for i, coeff in enumerate(poly):
        if coeff == 0:
            continue
        if i == 0:
            term = f"{coeff}"
        elif i == 1:
            term = f"{var}" if coeff == 1 else f"{coeff}{var}"
        else:
            term = f"{var}^{i}" if coeff == 1 else f"{coeff}{var}^{i}"
        terms.append(term)
    if not terms:
        return "0"
    return "+".join(reversed(terms))

def add_elements(a, b, p):
    """
    Add two elements in Z_p[x]/(mod_poly). Since each element is represented as a tuple
    of coefficients (of fixed length), addition is just coordinate-wise mod p.
    """
    return tuple((a[i] + b[i]) % p for i in range(len(a)))

def print_addition_table_tabulate(p, mod_poly, var="x"):
    """
    Prints the addition table for the quotient ring Z_p[x] / (mod_poly)
    in a neat grid format using the tabulate library.
    The header row and the first column are printed in bold.

    Parameters:
      - p: a prime number (the modulus for the coefficients)
      - mod_poly: the modulus polynomial, given as a list of coefficients
                  [constant, coeff_of_x, coeff_of_x^2, ..., coeff_of_x^n]
                  (It is assumed that mod_poly is monic.)
      - var: the variable name to use in printing the polynomial (default "x")
    """
    n = len(mod_poly) - 1  # The elements are polynomials of degree < n
    elements = [tuple(poly) for poly in itertools.product(range(p), repeat=n)]
    names = {elem: poly_to_str(elem, var) for elem in elements}

    # Build header with bold formatting.
    header = [BOLD_START + "" + BOLD_END]
    header.extend([BOLD_START + names[e] + BOLD_END for e in elements])

    table = []
    for e in elements:
        # Bold the row header.
        row = [BOLD_START + names[e] + BOLD_END]
        for f in elements:
            summ = add_elements(e, f, p)
            row.append(names[summ])
        table.append(row)

    mod_poly_str = poly_to_str(mod_poly, var)
    print(f"Addition Table for Z_{p}[{var}] / ({mod_poly_str}):")
    print(tabulate(table, headers=header, tablefmt="grid"))

# Example usage:
# For example, for GF(2)[x]/(x^2+1), we have p=2 and mod_poly = [1, 0, 1]
print_addition_table_tabulate(2, [1, 1, 0, 1], var="x")

# You can change p and mod_poly to work with other rings as needed.


Addition Table for Z_2[x] / (x^3+x+1):
+---------+---------+---------+---------+---------+---------+---------+---------+-----------+
|         | 0       | x^2     | x       | x^2+x   | 1       | x^2+1   | x+1     | x^2+x+1   |
+=========+=========+=========+=========+=========+=========+=========+=========+===========+
| 0       | 0       | x^2     | x       | x^2+x   | 1       | x^2+1   | x+1     | x^2+x+1   |
+---------+---------+---------+---------+---------+---------+---------+---------+-----------+
| x^2     | x^2     | 0       | x^2+x   | x       | x^2+1   | 1       | x^2+x+1 | x+1       |
+---------+---------+---------+---------+---------+---------+---------+---------+-----------+
| x       | x       | x^2+x   | 0       | x^2     | x+1     | x^2+x+1 | 1       | x^2+1     |
+---------+---------+---------+---------+---------+---------+---------+---------+-----------+
| x^2+x   | x^2+x   | x       | x^2     | 0       | x^2+x+1 | x+1     | x^2+1   | 1         |
+---------+---------+